In [1]:
import pandas as pd
import numpy as np
import abydos
from abydos import distance

# General Instruction

- Cleanup dataset based on the information that is given:
You need to clean the dataset according to the information that is given to you. This means that there are problems with the dataset that need to be fixed, and you should use the information given to you to determine what those problems are and how to fix them.

- Each case has different data quality problems, there will be hint and additional information that can help you understand the problem:
Each row in the dataset may have different data quality problems. There will be hints and additional information provided to help you understand what the specific problem is with each row.

- You can do any approach on cleaning the data, but you should clean the instructed column only:
You have the freedom to use any approach to clean the data, but you should only clean the instructed column. This means that you should not modify any other columns in the dataset, or add or remove any rows.

- Do not create new column or remove any column. Also do not create new row, or remove any row:-
You are not allowed to create new columns or remove any columns from the dataset. You are also not allowed to add or remove any rows.

- Each column will have a flag column something equivalent to <column\_name>\_flag. This column can be used to flag the row if you want to not include it to the downstream task. 0: safe_flag, 1: delete_flag, 2: null_flag (if you want to still include the row with null treatment). You can also add a new category but please add justification and explanation of the new category, there are three categories you can use:
safe_flag (0): this row is safe to use in downstream tasks
delete_flag (1): this row should be deleted and not used in downstream tasks
null_flag (2): this row can be included in downstream tasks but with null treatment.
You can also add a new category, but you need to provide a justification and an explanation for the new category. It is worth to note that the completeness of the dataset is also matter, so try not to flag to many things, and do your best to clean the values.

- For each data cleaning task, we have provided a function that represents the goal of the cleaning. For example, clean_duplicate_id(df) is the function for removing duplicate ID values. These functions take a DataFrame as input and return the cleaned version of the DataFrame.

    In each chunk of data cleaning task, you will see the following three parts:

    The clean_<name> function that performs the specific cleaning task.
    The execution of the cleaning function on the DataFrame.
    A checking part to help you evaluate the effectiveness of the cleaning.
    While you can create new cells and add additional code, the cleaning must be performed through the provided cleaning functions. You can adjust the order of the cleaning steps, but please try to move the whole chunks of code to avoid any errors.





# Purpose
The purpose of this dataset is to conduct exploratory analysis of the listings and create a prediction model for listing price using some columns from the dataset. This means that the dataset is intended to be used to explore the characteristics and features of the listings, and to build a model that can predict the price of a listing based on certain variables in the dataset. The goal is to gain insights into the factors that influence the price of a listing and to develop a model that can accurately predict listing prices based on those factors.

# Columns and Dataset Description
- id: a unique identifier for each listing.
- name: the name or title of the listing, as provided by the host.
- host_id: a unique identifier for each host.
- host_name: the name of the host who listed the property.
- neighbourhood_group: the larger geographic area in which the listing is located (e.g. a borough or group of neighborhoods).
- neighbourhood: the specific neighborhood in which the listing is located.
- latitude: the latitude coordinate of the listing.
- longitude: the longitude coordinate of the listing.
- room_type: the type of space that is being listed (e.g. an entire apartment, a private room, a shared room).
- price: the nightly price of the listing, in the currency specified in the dataset.
- minimum_nights: the minimum number of nights that a guest must book the listing for.
- number_of_reviews: the total number of reviews that the listing has received.
- last_review: the date of the most recent review of the listing.
- reviews_per_month: the average number of reviews per month that the listing has received.
- calculated_host_listings_count: the total number of listings that the host has on Airbnb.
- availability_365: the number of days per year that the listing is available for booking.
- number_of_reviews_ltm: the total number of reviews that the listing has received in the last 12 months.
- license: a license number for the listing, if applicable (this column may not be present in all versions of the dataset).

Besides the columns above, there are columns pre-defined for flagging the rows based on particular data cleaning context:
- id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the id column (duplicate).
- host_id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the host_id column.
- neighbourhood_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the neighbourhood column.
- latitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the latitude column.
- longitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the longitude column.
- minimum_nights_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the minimum_nights column.
- number_of_reviews_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the number_of_reviews column.
- last_review_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the last_review column.
- room_type_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the room_type column.

# Load Data

In [2]:
airbnb_pd = pd.read_csv("chicago_vert_dataset.csv")

In [3]:
airbnb_pd.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license', 'id_flag',
       'host_id_flag', 'neighbourhood_flag', 'latitude_flag', 'longitude_flag',
       'minimum_nights_flag', 'number_of_reviews_flag', 'last_review_flag',
       'room_type_flag'],
      dtype='object')

In [4]:
airbnb_pd.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,license,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,10945,The Biddle House (#1),33004,At Home Inn,NaN,Lincoln Park,41.91183,-87.64,Entire home/apt,95.0,...,2209984,0,0,0,0,0,0,0,0,0
1,12140,Lincoln Park Guest House,46734,Sharon And Robert,NaN,bsncWlnPPark,41.92335,-87.64951,Private room,329.0,...,R20000055258,0,0,0,0,0,0,0,0,0
2,24833,Prime LincolnPark 1 Block Fullerton Express L ...,101521,Red,NaN,Lincoln Park,41.925961,-87.656364,Entire home/apt,64.0,...,City registration pending,0,0,0,0,0,0,0,0,0
3,25879,2/1 One Block to Fullerton L Red Line Deck & ...,101521,Red,NaN,Lincoln Park,41.92693,-87.65753,Entire home/apt,94.0,...,City registration pending,0,0,0,0,0,0,0,0,0
4,207218,Historic Pullman Artist Flat - Artists & Explo...,1019125,Jb,NaN,Pullman,41.6883,-87.60892,Entire home/apt,100.0,...,R21000073121,0,0,0,0,0,0,0,0,0


# cleanup duplicate id
The ID column must contain unique values. If there are any duplicate values in this column, you will need to take action to ensure that each ID is unique. You can do this by either fixing the duplicates (if you want to keep them) or by flagging them for removal (1) using the id_flag column.

In [5]:
def clean_duplicate_id(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
    
    return df

In [6]:
airbnb_pd = clean_duplicate_id(airbnb_pd)

not yet have implementation


# Duplicate IDS checking 
To ensure that all ID values in the dataset are unique, you should check for duplicate IDs. When you run the query to check for duplicates, there should be no rows returned, indicating that there are no duplicate ID values present in the dataset.

In [7]:
dup_ids = airbnb_pd[airbnb_pd.id_flag==0]
dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
dup_ids = dup_ids[dup_ids.name>1]
dup_ids

,id,name
10,507517,2
14,697634,2
27,1321332,2
103,5980645,2
153,8485642,2
...,...,...
3241,761814267868760921,2
3255,766299554796204610,2
3264,769530444417979073,2
3272,771438179656564082,2


# cleanup inconsistent host id
Each host_id value in the dataset should be associated with only one host_name. However, there may be inconsistencies in the dataset where a host_id is associated with different host_name values.

To clean this up, you can either change the host_name value to a consistent value based on information in the dataset, or flag the host_id_flag column to indicate that the row should be removed from downstream tasks.

For example, if you find that a host_id is associated with multiple host_name values, you may want to investigate further to determine which host_name is correct. If one of the host_name values is clearly incorrect (e.g., a misspelling or a name that does not match the owner of the property), you could update the host_name value to the correct value.

Alternatively, if you cannot determine the correct host_name value, or if you want to exclude the row from downstream tasks for other reasons, you can flag the host_id_flag column with a value of 1 to indicate that the row should be removed.

In [8]:
def clean_host_id(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
    
    return df

In [9]:
airbnb_pd = clean_host_id(airbnb_pd)

not yet have implementation


# Inconsistent Host ID checking 

This query should return zero rows once you implement the cleaning process

In [10]:
dup_host_id = airbnb_pd[airbnb_pd.host_id_flag==0]
dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()
dup_host_id[dup_host_id["id"]>1]

,host_id,id
9,76244,2
10,100179,2
13,209564,2
17,483146,2
33,871911,2
...,...,...
1451,483081067,2
1455,485063892,2
1460,487492125,2
1461,488113016,2


# cleanup neighbourhood
The neighbourhood column in the dataset should contain values that match the neighbourhoods defined in the official neighbourhood_list. However, there may be some values in the neighbourhood column that are incorrect due to errors or noise in the data.

To clean up the neighbourhood column, you can try to match each value in the column to a valid neighbourhood in the neighbourhood_list using a string distance function such as abydos. If you can successfully match a value in the neighbourhood column to a neighbourhood in the neighbourhood_list, you can replace the value in the dataset with the correct neighbourhood name.

However, if you are unsure about how to clean up a particular value in the neighbourhood column, or if you cannot match the value to a valid neighbourhood in the neighbourhood_list, you can flag the row for deletion by setting the neighbourhood_flag column to a value of 1. If the value in the neighbourhood column is null and you cannot make a determination based on other information in the dataset, you can set the neighbourhood_flag column to a value of 2 to indicate that the row should be included but the neighbourhood value is null.

You can also use the latitude and longitude columns in the dataset to help match values in the neighbourhood column to valid neighbourhoods in the neighbourhood_list. However, you should be aware that the latitude and longitude values may also contain errors or noise, so you should exercise caution when using these columns to clean up the neighbourhood column.

In [11]:
neighbourhood_list = [ 'Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View',    'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater',    'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park',    'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side',    'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood',    'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale',    'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park',    'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore',    'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa',    'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park',    'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn',    'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights',    'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn',    'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland',    'West Englewood', 'Calumet Heights', 'Auburn Gresham', 'Fuller Park',    'Avalon Park', 'Burnside', 'Clearing', 'Gage Park', 'West Pullman',    'Washington Heights', 'East Side']
print(neighbourhood_list)

['Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View', 'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater', 'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park', 'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side', 'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood', 'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale', 'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park', 'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore', 'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa', 'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park', 'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn', 'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights', 'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn', 'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland', 'West Englewood', 'Calu

In [12]:
def clean_neighbourhood(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [13]:
airbnb_pd = clean_neighbourhood(airbnb_pd)

not yet have implementation


# Neighbourhood checking

This query should return zero rows once you implement the cleaning process

In [14]:
neighbourhood_check = airbnb_pd[airbnb_pd.neighbourhood_flag==0]
neighbourhood_check = neighbourhood_check[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
neighbourhood_check[["id","neighbourhood"]]

,id,neighbourhood
1,12140,bsncWlnPPark
6,220333,buqlman
9,350347,LakP Viel
25,1027463,Jorth DawRJale
28,1185749,Near iesk Sidk
...,...,...
3484,779793471889537254,NearqSCrth SXde
3488,780422673728726777,Near NortLESitK
3490,781904321701004603,Sakk View
3495,782446409146218784,Near egrth SiXe


# cleanup latitude and longitude
The latitude and longitude values in the dataset must fall within the range of -90 to +90 for latitude and -180 to +180 for longitude to ensure that they meet the criteria for analysis. We have provided a check number function to validate the latitude and longitude columns. Any values outside of these ranges should be cleaned to meet the criteria.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting latitude_flag or longitude_flag to 1 or 2, respectively.

In [15]:
def check_number(x,start=-90,end=90):
    try:
        temp_x = float(x)
        if start <= temp_x <= end:
            return True
    except:
        return False

In [16]:
def clean_latitude(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [17]:
airbnb_pd = clean_latitude(airbnb_pd)

not yet have implementation


# Latitude checking

This query should return zero rows once you implement the cleaning process

In [18]:
lat_check_pd = airbnb_pd[airbnb_pd.latitude_flag==0]
lat_check_pd = lat_check_pd[lat_check_pd.latitude.apply(lambda x:check_number(x,-90,90))==False]
lat_check_pd[["id","latitude"]]

,id,latitude
14,668262,4xY1.93173
58,2695259,jw41.93227
71,3434136,q41.94E02
85,4150121,41.m94D966
93,4852830,41nF.91221
...,...,...
3414,765870413335885635,41.tJ89123
3417,766299554796204610,41.A7769C88Y1
3418,766299554796204610,41.A7769C88Y1
3481,778656978525533480,41.7c668x5


In [19]:
def clean_longitude(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [20]:
airbnb_pd = clean_longitude(airbnb_pd)

not yet have implementation


# Longitude checking

This query should return zero rows once you implement the cleaning process

In [21]:
lon_check_pd = airbnb_pd[airbnb_pd.longitude_flag==0]
lon_check_pd = lon_check_pd[lon_check_pd.longitude.apply(lambda x:check_number(x,-180,180))==False]
lon_check_pd[["id","longitude"]]

,id,longitude
13,640841,I-M87.6712g8
28,1185749,R-87.eL68305
29,1185749,R-87.eL68305
50,2103986,-87.6p 3O923
65,3031898,-87.m5j69k51
...,...,...
3433,770376102705815314,-8bs7.63U845
3459,773908539670296526,-d8d7.64M7N9 7935767211
3464,775296271806256924,-8s7C.6894H9
3474,777038375268689288,-P87.L6023R2


# cleanup room type
The "room_type" column in the dataset should contain one of the values defined in the list of allowed_room_type provided by the authority: ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']. Any value outside of this list needs to be adjusted to one of the allowed values.

If you are unsure about how to adjust the value or cannot find a suitable value, you can flag the row for deletion by setting the value of room_type_flag to 1. If the "room_type" column has a null value and you cannot decide on an appropriate value, you can set the value of room_type_flag to 2.

In [22]:
allowed_room_type = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']

In [23]:
def clean_room_type(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [24]:
airbnb_pd = clean_room_type(airbnb_pd)

not yet have implementation


# room_type checking

This query should return zero rows once you implement the cleaning process

In [25]:
room_type_pd = airbnb_pd[airbnb_pd.room_type_flag==0]
room_type_pd = room_type_pd[room_type_pd.room_type.apply(lambda x: x not in allowed_room_type)]
room_type_pd[["id","room_type"]]

,id,room_type
10,507517,Entire home
11,507517,Entire home
15,697634,Entire home
16,697634,Entire home
20,887038,Entire home
...,...,...
3493,782137041139805370,Entire home
3499,783954262708483730,Entire home
3504,785426440046865418,Entire home
3505,785426710087480770,Entire home


# cleanup minimum_nights and number_of_reviews

The columns "minimum_nights" and "number_of_reviews" should both be integer values. "minimum_nights" should be a value between 1 and the number of days in a year (365), while "number_of_reviews" should be a value between 0 and 999999.

To check if these columns meet the criteria, we have provided a "check_integer" function. Any values that do not meet the criteria should be cleaned to meet the criteria for analysis.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting "minimum_nights_flag" or "number_of_reviews_flag" to 1 or 2, respectively.

In [26]:
def check_integer(x,start=1,end=365):
    try:
        temp_x = int(x)
        if start <= temp_x <= end:
            return True
    except:
        return False

In [27]:
def clean_minimum_nights(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [28]:
airbnb_pd = clean_minimum_nights(airbnb_pd)

not yet have implementation


# Minimum nights checking

This query should return zero rows once you implement the cleaning process

In [29]:
min_check_pd = airbnb_pd[airbnb_pd.minimum_nights_flag==0]
min_check_pd = min_check_pd[min_check_pd.minimum_nights.apply(lambda x:check_integer(x,1,365))==False]
min_check_pd[["id","minimum_nights"]]

,id,minimum_nights
6,220333,3B2
198,10227627,NaN
223,11609854,3y2
301,14382622,3o2
311,15114450,NaN
...,...,...
3443,772237002816161823,3c2
3444,772274762187370032,a32
3476,777399577291286339,3c2
3477,777399577291286339,3c2


In [30]:
def clean_number_of_reviews(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [31]:
airbnb_pd = clean_number_of_reviews(airbnb_pd)

not yet have implementation


# Clean number of reviews checking

This query should return zero rows once you implement the cleaning process

In [32]:
min_check_pd = airbnb_pd[airbnb_pd.number_of_reviews_flag==0]
min_check_pd = min_check_pd[min_check_pd.number_of_reviews.apply(lambda x:check_integer(x,1,999999))==False]
min_check_pd[["id","number_of_reviews"]]

,id,number_of_reviews
5,207351,4X4
29,1185749,NaN
53,2305230,U394
59,2730613,1L47
66,3068787,12l5
...,...,...
2962,690674302685751258,a14
2963,690674302685751258,a14
3276,742826464067294694,1V0
3398,761033002793394902,NaN


# cleanup last_review

The "last_review" column should be in the format of ISO-date (yyyy-mm-dd). We have provided a "check_date" function to verify the date format.

If a value is outside the date format or is null and you are unsure how to handle it, you can flag the row for deletion by setting the "last_review_flag" to 1 or 2.


In [33]:
from datetime import datetime
def check_date(x,fmt="%Y-%m-%d"):
    try:
        datetime.strptime(x,fmt)
        return True
    except:
        return False

In [34]:
def clean_last_reviews(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [35]:
airbnb_pd = clean_last_reviews(airbnb_pd)

not yet have implementation


# Last Review checking

This query should return zero rows once you implement the cleaning process

In [36]:
last_review_check_pd = airbnb_pd[airbnb_pd.last_review_flag==0]
last_review_check_pd = last_review_check_pd[last_review_check_pd.last_review.apply(lambda x:check_date(x))==False]
last_review_check_pd[["id","last_review"]]

,id,last_review
3,25879,"November 13, 2022"
24,1020810,NaN
43,1769731,"November 06, 2022"
57,2611538,"August 22, 2022"
59,2730613,"November 29, 2022"
...,...,...
3507,785428954368670590,NaN
3508,785429252974057580,NaN
3509,785429618431075490,NaN
3510,785429886476112156,NaN


# columns that potentially will be used for analysis:
id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,price

In [37]:
columns_used = ["id","name","host_id","host_name",
                         "neighbourhood","latitude","longitude",
                         "room_type","minimum_nights","number_of_reviews","last_review","price"]

In [38]:
airbnb_pd[columns_used]

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,price
0,10945,The Biddle House (#1),33004,At Home Inn,Lincoln Park,41.91183,-87.64,Entire home/apt,4,59,2022-11-26,95.0
1,12140,Lincoln Park Guest House,46734,Sharon And Robert,bsncWlnPPark,41.92335,-87.64951,Private room,2,13,2022-09-12,329.0
2,24833,Prime LincolnPark 1 Block Fullerton Express L ...,101521,Red,Lincoln Park,41.925961,-87.656364,Entire home/apt,32,41,2022-11-12,64.0
3,25879,2/1 One Block to Fullerton L Red Line Deck & ...,101521,Red,Lincoln Park,41.92693,-87.65753,Entire home/apt,32,50,"November 13, 2022",94.0
4,207218,Historic Pullman Artist Flat - Artists & Explo...,1019125,Jb,Pullman,41.6883,-87.60892,Entire home/apt,2,298,2022-11-13,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3507,785428954368670590,"River North 1br w/ gym, pool & roof, nr Riverwalk",107434423,Blueground,Near North Side,41Vuv.890516,-87.635955,Entire home/apt,32,0,NaN,204.0
3508,785429252974057580,"Streeterville 2br w/ pool & gym, nr Riverwalk",107434423,Blueground,Near North Side,41.8911616,-87.62226299999999,Entire home/apt,32,0,NaN,130.0
3509,785429618431075490,"River North 1br w/ gym, lounge & roof nr River...",107434423,Blueground,Near North Side,41.8945286,-87.63340520000003,Entire home,32,0,NaN,115.0
3510,785429886476112156,"River North 1br w/ gym, lounge & roof nr River...",107434423,Blueground,Near North Side,41.8945286,-87.63340520000003,Entire home/apt,32,0,NaN,115.0
